In [ ]:
#| default_exp argument_validators

In [ ]:
#| export
from typing import Protocol, Any, runtime_checkable
from pymoq.core import AnyCallable

In [ ]:
#| hide
from nbdev.showdoc import *

# Validators

> Collection of methods to validate specific call arguments.

Goal: Evaluate whether a call like `f(1,"s")` matches any signature-pattern. A signature pattern might be defined like `f(1, str)`. This should match any call that passes the exact value one for the first argument and any object of type str in the second.

## Argument validators

We break the task down to validating a single argument. The signature of such an ArgumentValidator should look like:

In [ ]:
#| export
@runtime_checkable
class ArgumentValidator(Protocol):
    "Interface for all argument validators"
    def is_valid(self, argument: Any) -> bool:
        ...

The most flexibility can be achieved by constructing an ArgumentValidator that evaluates an arbitrary function:

In [ ]:
#| export
class ArgumentFunctionValidator:
    "Validate an argument by evaluating an arbitrary function"
    def __init__(self, func: AnyCallable[bool]):
        self._func = func
        
    def is_valid(self, argument: Any) -> bool:
        return self._func(argument)
    
assert isinstance(ArgumentFunctionValidator, ArgumentValidator)

This could now be used like:

In [ ]:
any_int = ArgumentFunctionValidator(lambda v: isinstance(v, int))

In [ ]:
assert any_int.is_valid(1)
assert not any_int.is_valid(1.1)
assert not any_int.is_valid("string")

In later stages there should be convenience methods around creating such argument validators. E.g. `from_type(some_type)` for making the above easier.

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()